<a href="https://colab.research.google.com/github/antopeza/Final-Project-Brainster/blob/main/NEW_code_Doc2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from numpy import random
from numpy import vstack
import matplotlib.pyplot as plt
import os 

import re

import string
from string import punctuation


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import xgboost as xgb

import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, word_tokenize

import gensim
from gensim.parsing.preprocessing import remove_stopwords

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential, load_model
from keras.layers import Input, Embedding, LSTM, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

from imblearn.over_sampling import SMOTE
from collections import Counter

#from scikeras.wrappers import KerasClassifier

import pickle
from pickle import dump
from pickle import load

from imblearn.over_sampling import SMOTE
from collections import Counter

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive 
drive.mount('/drive')

Mounted at /drive


In [4]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
stops = set(stopwords.words('english'))
#print(stops)

In [6]:
stemmer =  PorterStemmer()
lemmer = WordNetLemmatizer()

In [7]:
#df_raw = pd.read_csv(path)
df_raw = pd.read_csv('/drive/My Drive/MACHINE LEARNING/FINAL PROJECT/train.csv')

In [8]:
print('shape of df  = ', df_raw.shape)
print()
df_raw.head(2)

shape of df  =  (404290, 6)



,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0


In [8]:
df_raw[df_raw.isnull().any(axis=1)]

,id,qid1,qid2,question1,question2,is_duplicate
105780,105780,174363,174364,How can I develop android app?,NaN,0
201841,201841,303951,174364,How can I create an Android app?,NaN,0
363362,363362,493340,493341,NaN,My Chinese name is Haichao Yu. What English na...,0


In [8]:
df_raw = df_raw.fillna("")
df_raw.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [9]:
# randomly selected samples from df for the new df to perform operations
df = df_raw.sample(5000)
df = df[['question1', 'question2', 'is_duplicate']]
print(df.shape)

(5000, 3)


In [10]:
df_copy = df.copy()

# Clean Data

In [10]:
def get_wordnet_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    if tag.startswith("J"):
        return wordnet.ADJ
    if tag.startswith("V"):
        return wordnet.VERB
    if tag.startswith("R"):
        return wordnet.ADV
    return wordnet.NOUN

def clean_sentence(sentence, stopwords = False, lemmatize = False, stem = False):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)  # removing all caracters that are not alpha numeric
    #sentence = re.sub(r'\s{2,}', ' ', sentence
   
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))  # remove punctuation
    
    if stopwords:
        sentence = remove_stopwords(sentence)  
        
    if lemmatize:
        nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
        wn_tagged = map(lambda x: (x[0], get_wordnet_pos(x[1])), nltk_tagged)
        res_words = []
        for word, tag in wn_tagged:
            if tag is None:
                res_words.append(word)
            else :
                res_words.append(lemmer.lemmatize(word, tag))
        sentence = " ".join(res_words)
        
    if stem:
        sent_stemmed = ''
        for word in sentence.split():
            sent_stemmed += ' '+ stemmer.stem(word)
        sentence = sent_stemmed
   
    return sentence

def get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False):
    #sents1 = dfq[['question1']]  
    #sents2 = dfq[['question2']]
    cleaned_sentences1 = []
    cleaned_sentences2 = []
    
    for index, row in df.iterrows():
        # print (index, row)
        cleaned1 = clean_sentence(row['question1'], stopwords, lemmatize, stem) 
        cleaned2 = clean_sentence(row['question2'], stopwords, lemmatize, stem)

        cleaned_sentences1.append(cleaned1)
        cleaned_sentences2.append(cleaned2)
    return cleaned_sentences1, cleaned_sentences2

In [11]:
def to_df(q1cleaned, q2cleaned, df):
    '''cleaned sentences to df'''
    X_temp1 = pd.DataFrame()
    X_temp2 = pd.DataFrame()
    X_temp1['q1'] = pd.DataFrame(q1cleaned, index = df.index)
    X_temp2['q2'] = pd.DataFrame(q2cleaned, index = df.index)
    X_temp = pd.concat([X_temp1, X_temp2], axis = 1)
    X_temp['is_duplicate'] = df_raw['is_duplicate']
    return X_temp

#### With Stopwords, lemmatized, not stemmed

In [12]:
# With stopwords lemmatized, not stemmed
q1_sw_lemma, q2_sw_lemma = get_cleaned_senteces(df, stopwords = False, lemmatize = True, stem = False)
assert len(q1_sw_lemma) == len(q2_sw_lemma)
df_sw_lemma = to_df(q1_sw_lemma, q2_sw_lemma, df)
df_sw_lemma.shape

(5000, 3)

#### With Stopwords, stemmed, not lemmatized 

In [13]:
q1_sw_stem, q2_sw_stem = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = True)
assert len(q1_sw_stem) == len(q2_sw_stem)
df_sw_stem = to_df(q1_sw_stem, q2_sw_stem, df)
df_sw_stem.shape

(5000, 3)

#### With stopwords, not lemmatized nor stemmed

In [15]:
q1_sw, q2_sw = get_cleaned_senteces(df, stopwords = False, lemmatize = False, stem = False)
assert len(q1_sw) == len(q2_sw)
df_sw = to_df(q1_sw, q2_sw, df)
df_sw.shape

(5000, 3)

#### Without Stopwords, lemmatized, not stemmed


In [16]:
q1_lemma, q2_lemma = get_cleaned_senteces(df, stopwords = True, lemmatize = True, stem = False)
assert len(q1_lemma) == len(q2_lemma)
df_lemma = to_df(q1_lemma, q2_lemma, df)
df_lemma.shape

(5000, 3)

#### Without Stopwords, stemmed

In [17]:
q1_stem, q2_stem = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = True)
assert len(q1_stem) == len(q2_stem)
df_stem = to_df(q1_stem, q2_stem, df)
df_stem.shape

(5000, 3)

#### Without Stopwords, not lemmatized nor stemmed

In [20]:
q1_out, q2_out = get_cleaned_senteces(df, stopwords = True, lemmatize = False, stem = False)
assert len(q1_out) == len(q2_out)
df_out = to_df(q1_out, q2_out, df)
df_out.shape

(5000, 3)

In [21]:
display ('With stopwords, without lemma and stem', df_sw.iloc[[22]])
display ('With stopwords, lemmatized and without stem', df_sw_lemma.iloc[[22]])
display ('With stopwords, stemmed and without lemma', df_sw_stem.iloc[[22]])
display ('Without stopwords, lemma and stem', df_out.iloc[[22]])
display ('Without stopwords and stem, lemmatized', df_lemma.iloc[[22]])
display ('Without stopwords and lemma, stemmed', df_stem.iloc[[22]])

'With stopwords, without lemma and stem'

,q1,q2,is_duplicate
272872,what does it feel like to be on quora,what does it feel like to post on quora,1


'With stopwords, lemmatized and without stem'

,q1,q2,is_duplicate
272872,what do it feel like to be on quora,what do it feel like to post on quora,1


'With stopwords, stemmed and without lemma'

...

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and stem, lemmatized'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


'Without stopwords and lemma, stemmed'

,q1,q2,is_duplicate
272872,feel like quora,feel like post quora,1


In [119]:
# print('Value counts:\n', X_sw_train['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_train['is_duplicate'].value_counts()/X_sw_train['is_duplicate'].count()*100)
# X_sw_train['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of train labels', color = 'purple', size = 12)
# plt.show()

In [120]:
# print('Value counts:\n', X_sw_test['is_duplicate'].value_counts())
# print('\nPercent of labels value:\n', X_sw_test['is_duplicate'].value_counts()/X_sw_test['is_duplicate'].count()*100)
# X_sw_test['is_duplicate'].value_counts().plot(kind = 'bar')
# plt.title('\nDistribution of test labels', color = 'purple', size = 12)
# plt.show()

In [121]:
# display (X_sw_train.head(2))
# display(X_sw_test.head(2))
# display(df_clean.head(2))

In [44]:
#X_sw_test[X_sw_test.isnull().any(axis=1)]

# *TOKENIZATION

In [22]:
def tokenization(text):
    tokens = re.split(' ',text)
    return tokens
def token(text):
    text = text.apply(lambda x: tokenization(x))
    return text

In [122]:
df_lemma['q1'] = df_lemma['q1'].pipe(token) 
df_lemma['q2'] = df_lemma['q2'].pipe(token) 
df_lemma.head(2)

,q1,q2,is_duplicate
179733,"[idea, decorate, project, business, study]","[study, start, successful, business, nice, idea]",0
169496,"[country, candy, corn, come]","[like, eat, candy]",0


In [ ]:
df_sw_lemma['q1'] = df_sw_lemma['q1'].pipe(token) 
df_sw_lemma['q2'] = df_sw_lemma['q2'].pipe(token) 
df_sw_lemma.head(2)

In [ ]:
df_sw_stem['q1'] = df_sw_stem['q1'].pipe(token) 
df_sw_stem['q2'] = df_sw_stem['q2'].pipe(token) 
df_sw_stem.head(2)

In [58]:
df_stem['q1'] = df_stem['q1'].pipe(token) 
df_stem['q2'] = df_stem['q2'].pipe(token) 
df_stem.head(2)

,q1,q2,is_duplicate
327246,"[, can, i, regist, my, startup, compani, in, t...","[, can, i, start, a, compani, in, the, us, wit...",1
26328,"[, what, is, a, good, entrylevel, job, in, the...","[, what, are, the, best, entri, level, job, in...",0


In [ ]:
df_sw['q1'] = df_sw['q1'].pipe(token) 
df_sw['q2'] = df_sw['q2'].pipe(token) 
df_stem.head(2)

In [23]:
df_out['q1'] = df_out['q1'].pipe(token) 
df_out['q2'] = df_out['q2'].pipe(token) 
df_out.head(2)

,q1,q2,is_duplicate
179733,"[ideas, decorate, project, business, studies]","[study, start, successful, business, nice, idea]",0
169496,"[country, candy, corn, come]","[like, eating, candy]",0


#### Doc2Vec Without Stopwords, not stemmed not lemmatized

In [26]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
def doc2vec(df):
  q1 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q1'])]
  q2 = [TaggedDocument(d, [i]) for i, d in enumerate(df['q2'])]
  return q1, q2

In [50]:
def model_doc2vec(q1, vector, window, epochs):
  q1 = Doc2Vec(q1, dm = 1, vector_size = vector, window = window, min_count = 2, epochs = epochs)
  return q1

### Apply Doc2Vec

In [123]:
#lemmatized, without stopwords
q1_lemma_d2v, q2_lemma_d2v = doc2vec(df_lemma)
assert len(q1_lemma_d2v) == len(q2_lemma_d2v)

In [ ]:
#lemmatized with stopwords
q1_sw_lemma_d2v, q2_sw_lemma_d2v = doc2vec(df_sw_lemma)
assert len(q1_sw_lemma_d2v) == len(q2_sw_lemma_d2v)

In [55]:
#Stemmed without stopwords
q1_stem_d2v, q2_stem_d2v = doc2vec(df_stem)
assert len(q1_stem_d2v) == len(q2_stem_d2v)

In [ ]:
#Stemmed with stopwords
q1_sw_stem_d2v, q2_sw_stem_d2v = doc2vec(df_sw_stem)
assert len(q1_sw_stem_d2v) == len(q2_sw_stem_d2v)

In [ ]:
#With Stopwords, not lemmatized, not stemmed
q1_sw_d2v, q2_sw_d2v = doc2vec(df_sw)
assert len(q1_sw_d2v) == len(q2_sw_d2v)

In [ ]:
#Without stopwords, not lemmatized, not stemmed 
q1_out_d2v, q2_out_d2v = doc2vec(df_out)
assert len(q1_out_d2v) == len(q2_out_d2v)

### Activating Doc2Vec model

In [124]:
#PARAMETER FOR DOC2VEC MODEL 
vector = 100
window = 3
epochs = 20

In [125]:
#lemmatized, without stopwords
X_q1_lemma_d2v = model_doc2vec(q1_lemma_d2v, vector, window, epochs)
X_q2_lemma_d2v = model_doc2vec(q2_lemma_d2v, vector, window, epochs)

In [ ]:
#lemmatized, with stopwords
X_q1_sw_lemma_d2v = model_doc2vec(q1_sw_lemma_d2v, vector, window, epochs)
X_q2_sw_lemma_d2v = model_doc2vec(q2_sw_lemma_d2v, vector, window, epochs)

In [47]:
#Stemmed, without stopwords
X_q1_stem_d2v = model_doc2vec(q1_stem_d2v, vector, window, epochs)
X_q2_stem_d2v = model_doc2vec(q2_stem_d2v, vector, window, epochs)

[TaggedDocument(words=['ideas', 'decorate', 'project', 'business', 'studies'], tags=[0]),
 TaggedDocument(words=['country', 'candy', 'corn', 'come'], tags=[1])]

In [31]:
#Stemmed, with stopwords
X_q1_sw_stem_d2v = model_doc2vec(q1_sw_stem_d2v, vector, window, epochs)
X_q2_sw_stem_d2v = model_doc2vec(q2_sw_stem_d2v, vector, window, epochs)

[TaggedDocument(words='what are some ideas to decorate my project of business studies', tags=[0]),
 TaggedDocument(words='which country did candy corn come from', tags=[1])]

In [87]:
#With Stopwords, not lemmatized, not stemmed
X_q1_sw_d2v = model_doc2vec(q1_sw_d2v, vector, window, epochs)
X_q2_sw_d2v = model_doc2vec(q2_sw_d2v, vector, window, epochs)

In [81]:
#Without stopwords, not lemmatized, not stemmed 
X_q1_out_d2v = model_doc2vec(q1_out_d2v, vector, window, epochs)
X_q2_out_d2v = model_doc2vec(q2_out_d2v, vector, window, epochs)


#### Convert to array 

In [135]:
# Lemmatized, with stopwords, not stemmed

size = len(df) - 1
arr_q1_lemma = X_q1_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_lemma_d2v[i+1]
  arr_q1_lemma = np.vstack([arr_q1_lemma, arrT])

arr_q2_lemma = X_q2_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_lemma_d2v[i+1]
  arr_q2_lemma = np.vstack([arr_q2_lemma, arrT])

assert arr_q1_lemma.shape == arr_q2_lemma.shape

#arr_q1_lemma = create_arrays(X_q1_lemma_d2v)
#arr_q2_lemma = create_arrays(X_q2_lemma_d2v)

In [ ]:
# Lemmatized, without stopwords, not stemmed

size = len(df) - 1
arr_q1_sw_lemma = X_q1_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_lemma_d2v[i+1]
  arr_q1_sw_lemma = np.vstack([arr_q1_sw_lemma, arrT])

arr_q2_sw_lemma = X_q2_sw_lemma_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_lemma_d2v[i+1]
  arr_q2_sw_lemma = np.vstack([arr_q2_sw_lemma, arrT])

assert arr_q1_sw_lemma.shape == arr_q2_sw_lemma.shape

# arr_q1_sw_lemma = create_arrays(X_q1_sw_lemma_d2v)
# arr_q2_sw_lemma = create_arrays(X_q2_sw_lemma_d2v) 

In [ ]:
# Stemmed, with stopwords, not lemmatized

size = len(df) - 1
arr_q1_stem = X_q1_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_stem_d2v[i+1]
  arr_q1_stem = np.vstack([arr_q1_stem, arrT])

arr_q2_stem = X_q2_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_stem_d2v[i+1]
  arr_q2_stem = np.vstack([arr_q2_stem, arrT])

assert arr_q1_stem.shape == arr_q2_stem.shape

# arr_q1_stem = create_arrays(X_q1_stem_d2v)
# arr_q2_stem = create_arrays(X_q2_stem_d2v)

In [ ]:
# Stemmed, without stopwords, not lemmatized 

size = len(df) - 1
arr_q1_sw_stem = X_q1_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_stem_d2v[i+1]
  arr_q1_sw_stem = np.vstack([arr_q1_sw_stem, arrT])

arr_q2_sw_stem = X_q2_sw_stem_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_stem_d2v[i+1]
  arr_q2_sw_stem = np.vstack([arr_q2_sw_stem, arrT])

assert arr_q1_sw_stem.shape == arr_q2_sw_stem.shape

# arr_q1_sw_stem = create_arrays(X_q1_sw_stem_d2v)
# arr_q2_sw_stem = create_arrays(X_q2_sw_stem_d2v)

In [83]:
# Without stopwords, not lemmatized, not stemmed
size = len(df) - 1
arr_q1_out = X_q1_out_d2v[0] 

for i in range(size):
  arrT = X_q1_out_d2v[i+1]
  arr_q1_out = np.vstack([arr_q1_out, arrT])

arr_q2_out = X_q2_out_d2v[0] 

for i in range(size):
  arrT = X_q2_out_d2v[i+1]
  arr_q2_out = np.vstack([arr_q2_out, arrT])

assert arr_q1_out.shape == arr_q2_out.shape

In [99]:
# With stopword, not lemmatized, not stemmed

size = len(df) - 1
arr_q1_sw = X_q1_sw_d2v[0] 

for i in range(size):
  arrT = X_q1_sw_d2v[i+1]
  arr_q1_sw = np.vstack([arr_q1_sw, arrT])

arr_q2_sw = X_q2_sw_d2v[0] 

for i in range(size):
  arrT = X_q2_sw_d2v[i+1]
  arr_q2_sw = np.vstack([arr_q2_sw, arrT])

assert arr_q1_sw.shape == arr_q2_sw.shape

### Convert to dataframe 

In [89]:
def df_arr(arr_q1, arr_q2, df_one):
  tem1 = pd.DataFrame()
  tem2 = pd.DataFrame()
  temp1['q1'] = pd.DataFrame(arr_q1, index = df.index)
  temp2['q2'] = pd.DataFrame(arr_q2, index = df.index)
  X_temp = pd.concat([temp1, temp2], axis=1)
  X_temp['is_duplicate'] = df['is_duplicate']
  return X_temp

In [ ]:
q2_df_model2 = pd.DataFrame(q2_arr_st)
q2_df_model2 = pd.DataFrame(q2_arr_st)
X_model2 = pd.concat([q2_df_model2, q2_df_model2], axis=1)
y = df['is_duplicate']
y = np.array(y)
y_df = pd.DataFrame(y)
y_df.shape
df_model2 = X_model2.join(y_df, lsuffix='_left', rsuffix='_right')
df_model2.shape

In [ ]:
def to_df(q1cleaned, q2cleaned, df):
    '''cleaned sentences to df'''
    X_temp1 = pd.DataFrame()
    X_temp2 = pd.DataFrame()
    X_temp1['q1'] = pd.DataFrame(q1cleaned, index = df.index)
    X_temp2['q2'] = pd.DataFrame(q2cleaned, index = df.index)
    X_temp = pd.concat([X_temp1, X_temp2], axis = 1)
    X_temp['is_duplicate'] = df['is_duplicate']
    return X_temp

# DOC2VEC - STEMMED

In [108]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

q1_stem = [TaggedDocument(d, [i]) for i, d in enumerate(df_stem['q1'])]
q2_stem = [TaggedDocument(d, [i]) for i, d in enumerate(df_stem['q2'])]

In [109]:
q1_model_stem = Doc2Vec(q1_stem, dm = 0, vector_size = 8, window = 2, min_count = 1, epochs = 10)
q2_model_stem = Doc2Vec(q2_stem, dm = 0, vector_size = 8, window = 2, min_count = 1, epochs = 10)

In [63]:
# vector = model_stem.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
# print(vector)

[-0.02208794 -0.05290269  0.02206466  0.02093315 -0.00707981]


In [69]:
q1_arr_st = q1_model_stem[0] 

for i in range(2999):
  arrT = q1_model_stem[i+1]
  q1_arr_st = np.vstack([q1_arr_st, arrT])

#print(q1_arr)

q2_arr_st = q2_model_stem[0] 

for i in range(2999):
  arrT = q2_model_stem[i+1]
  q2_arr_st = np.vstack([q2_arr_st, arrT])

#print(q2_arr)

In [70]:
q2_df_model2 = pd.DataFrame(q2_arr_st)
q2_df_model2 = pd.DataFrame(q2_arr_st)
X_model2 = pd.concat([q2_df_model2, q2_df_model2], axis=1)
y = df['is_duplicate']
y = np.array(y)
y_df = pd.DataFrame(y)
y_df.shape
df_model2 = X_model2.join(y_df, lsuffix='_left', rsuffix='_right')
df_model2.shape

(3000, 17)

In [76]:
y= df_model2['0_right']
X_train_st, X_test_st, y_train_st, y_test_st = train_test_split(df_model2, y, stratify = y, test_size = 0.2, random_state = 0)

In [77]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    rf_param_grid = {'n_estimators':[200, 500, 800], 'min_samples_split':[3, 13], 'max_depth': [10, 30, None]}
    rf_clf = RandomForestClassifier(random_state = 0)
    
    rf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv = splits, factor = 2, scoring = 'accuracy', 
                                        max_resources = 5, aggressive_elimination = False, verbose = 0)
    rf_search.fit(X_train, y_train)
    
    rf_model = rf_search.best_estimator_
    
    y_pred = rf_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_model

In [78]:
acc_rf, precision_rf, recall_rf, f1_score_rf, log_loss_rf, y_pred_rf, berf  = rf_clf(X_train_st, X_test_st, y_train_st, y_test_st)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=200,
                       random_state=0)


In [79]:
report_rf = classification_report(y_pred_rf, y_test_st)
matrix_rf = confusion_matrix(y_pred_rf, y_test_st)
print('Classification Report - RF\n', report_rf)
print()
print('Confusion Matrix - RF\n', matrix_rf)

Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       390
           1       1.00      1.00      1.00       210

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600


Confusion Matrix - RF
 [[390   0]
 [  0 210]]


#otro intento doc2vec

In [89]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

X_sw_train_tag1 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q1'])]
X_sw_train_tag2 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q2'])]


In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

X_train_tag1 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q1'])]
X_train_tag2 = [TaggedDocument(d, [i]) for i, d in enumerate(X_cleaned['q2'])]

In [106]:
X_train_tag1[0:5]

[TaggedDocument(words='why do prophet muhammad marry 6 year old girl why dint he think to adopt her keep calm and answer', tags=[0]),
 TaggedDocument(words='how instagram be helpful', tags=[1]),
 TaggedDocument(words='be there an app that give you recipe base on what you have in the fridge', tags=[2]),
 TaggedDocument(words='what do the phrase shes get character mean', tags=[3]),
 TaggedDocument(words='what do lamb chop taste like', tags=[4])]

In [104]:
X_sw_test_tag1 = [TaggedDocument(d, [i]) for i, d in enumerate(X_sw_test['q1'])]
X_sw_test_tag2 = [TaggedDocument(d, [i]) for i, d in enumerate(X_sw_test['q2'])]

In [17]:
X_train_tagged = X_train_tag1 + X_train_tag2
#X_train_tagged

#X_test_tagged = X_sw_test_tag1 + X_sw_test_tag2


# Doc2Vec and similarity

In [18]:
model_doc2vec = Doc2Vec(vector_size =20, window=2, min_count=1, epochs=50)

In [19]:
model_doc2vec.build_vocab(X_train_tagged)

In [20]:
model_doc2vec.train(X_train_tagged, total_examples=model_doc2vec.corpus_count, epochs=model_doc2vec.epochs)

In [109]:
print(model_doc2vec.infer_vector(X_sw_test))

[ 0.00553507 -0.02057612 -0.00601494  0.01590716  0.00477619 -0.00957217
 -0.01354616  0.01898067 -0.00717559  0.00853975  0.00150559 -0.01450056
 -0.00628278 -0.02179538 -0.01993607 -0.02084691  0.00312932 -0.0190215
 -0.00581192  0.01894295]


In [90]:
#TEST THE MODEL ON X_sw_test

test = model_doc2vec.infer_vector(X_sw_test)
model_doc2vec.docvecs.most_similar(positive = [test])
#positive = List of sentences that contribute positively.


[(1607, 0.3674626350402832),
 (1454, 0.3508557975292206),
 (1876, 0.3459669351577759),
 (2316, 0.3180397152900696),
 (1868, 0.31006860733032227),
 (389, 0.3054264485836029),
 (1438, 0.28425800800323486),
 (1760, 0.272051066160202),
 (2109, 0.2675776481628418),
 (2054, 0.2670101821422577)]

# buen codigo para doc2vec cosine similarity

In [16]:
# import
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
q1_tag = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q1'])]
q2_tag = [TaggedDocument(d, [i]) for i, d in enumerate(df_clean['q2'])]
#tagged_data

In [17]:
q1_model = Doc2Vec(q1_tag, vector_size = 20, window = 2, min_count = 1, epochs = 100)
q2_model = Doc2Vec(q2_tag, vector_size = 20, window = 2, min_count = 1, epochs = 100)

In [31]:
q1_model[0]

array([ 0.5810266 ,  1.2083697 ,  0.9162264 , -0.0174998 , -0.15741944,
       -1.4726185 , -0.62223524,  0.5812599 ,  0.7438206 , -0.04712546,
       -0.8470433 ,  0.6188074 ,  0.6834843 ,  0.34443977, -0.66070366,
        0.66500264,  0.1121475 , -0.8982408 ,  0.18651539, -0.18247873],
      dtype=float32)

In [76]:
# #cosine similarity - to finish 
# test_doc = word_tokenize("What is the step by step guide to invest".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])


[(1555, 0.7962811589241028),
 (2760, 0.7377956509590149),
 (1810, 0.7373658418655396),
 (2765, 0.7061798572540283),
 (2729, 0.6963450312614441),
 (1806, 0.688534677028656),
 (1839, 0.685553789138794),
 (2233, 0.6750451326370239),
 (1854, 0.6687722206115723),
 (2190, 0.6662200093269348)]

buen codigo

In [19]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
q1_lemma = [TaggedDocument(d, [i]) for i, d in enumerate(df_lemma['q1'])]
q2_lemma = [TaggedDocument(d, [i]) for i, d in enumerate(df_lemma['q2'])]


In [85]:
model = Doc2Vec(tagged_data_lemma, vector_size = 10, window = 2, min_count = 1, epochs = 100)
model

In [81]:
# test_doc = word_tokenize("What is the step by step guide to invest".lower())
# test_doc_vector = model.infer_vector(test_doc)
# model.docvecs.most_similar(positive = [test_doc_vector])

[(1854, 0.8070089221000671),
 (42, 0.7843912839889526),
 (958, 0.779266357421875),
 (1183, 0.7622733116149902),
 (1839, 0.7594348192214966),
 (2007, 0.7579702138900757),
 (2760, 0.7547283172607422),
 (909, 0.7446797490119934),
 (2765, 0.7345895767211914),
 (1835, 0.732286810874939)]

#try svc

In [86]:
y= df['0_right']
X_train, X_test, y_train, y_test = train_test_split(df, y, stratify = y, test_size = 0.2, random_state = 0)

In [90]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
#X, y = make_classification(random_state=0, shuffle=False)
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)


RandomForestClassifier(max_depth=2, random_state=0)

In [38]:
from sklearn.model_selection import ShuffleSplit

In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV

In [91]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    rf_param_grid = {'n_estimators':[200, 500, 800], 'min_samples_split':[5, 15], 'max_depth': [70, 150, None]}
    rf_clf = RandomForestClassifier(random_state = 0)
    
    rf_bow_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv = splits, factor = 2, scoring = 'accuracy', 
                                        max_resources = 40, aggressive_elimination = True,verbose = 0)
    rf_bow_search.fit(X_train, y_train)
    
    rf_bow_model = rf_bow_search.best_estimator_
    
    y_pred = rf_bow_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_bow_model)
    #print('log loss', rf_loss_bow, '\nacc', rf_acc_bow)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_bow_model

In [97]:
acc_rf, precision_rf, recall_rf, f1_score_rf, log_loss_rf, y_pred_rf, berf  = rf_clf(X_train, X_test, y_train, y_test)

RandomForestClassifier(min_samples_split=15, n_estimators=500, random_state=0)


In [98]:
report_BoW_rf = classification_report(y_pred_rf, y_test)
matrix_BoW_rf = confusion_matrix(y_pred_rf, y_test)
print('Classification Report - RF\n', report_BoW_rf)
print()
print('Confusion Matrix - RF\n', matrix_BoW_rf)

Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       381
           1       1.00      1.00      1.00       219

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600


Confusion Matrix - RF
 [[381   0]
 [  0 219]]


#Doc2Vec - Lemmatized

In [93]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
q1_lemma = [TaggedDocument(d, [i]) for i, d in enumerate(df_lemma['q1'])]
q2_lemma = [TaggedDocument(d, [i]) for i, d in enumerate(df_lemma['q2'])]

In [94]:
#Model for question 1 and question 2 separately
q1_model_one = Doc2Vec(q1_lemma, dm = 0, vector_size = 10, window = 2, min_count = 1, epochs = 20)
q2_model_one = Doc2Vec(q2_lemma, dm = 0, vector_size = 10, window = 2, min_count = 1, epochs = 20)

In [95]:
q1_arr = q1_model_one[0] 

for i in range(2999):
  arrT = q1_model_one[i+1]
  q1_arr = np.vstack([q1_arr, arrT])

print(q1_arr)

q2_arr = q2_model_one[0] 

for i in range(2999):
  arrT = q2_model_one[i+1]
  q2_arr = np.vstack([q2_arr, arrT])

print(q2_arr)

[[ 0.10032979 -0.55578184 -0.06625117 ... -0.25241283 -0.05939401
  -0.7159061 ]
 [ 0.08974945 -0.5286851   0.02084151 ... -0.23568097  0.07601853
  -0.5445449 ]
 [ 0.06887603 -0.23924154 -0.04770829 ... -0.09055078  0.01026634
  -0.31425464]
 ...
 [ 0.06162458 -0.4878645  -0.11002222 ... -0.18940294  0.00814996
  -0.5836134 ]
 [ 0.05525352 -0.34166166 -0.06969745 ... -0.15237166  0.0567542
  -0.48107553]
 [ 0.10098049 -0.33037513 -0.0701501  ... -0.11666341 -0.00074098
  -0.4287096 ]]
[[-0.07451188 -0.24052733 -0.02674165 ... -0.16225918  0.06101755
  -0.6091608 ]
 [-0.06616969 -0.26921144  0.06970199 ... -0.1462597   0.14958256
  -0.40313926]
 [-0.02195901 -0.16725123 -0.00397086 ... -0.11239309  0.14229198
  -0.42865592]
 ...
 [-0.05998328 -0.29216203  0.04668297 ... -0.14245567  0.14026663
  -0.5836935 ]
 [-0.09857899 -0.2456381   0.01048574 ... -0.11982813  0.1782285
  -0.6047648 ]
 [ 0.00367951 -0.30113846  0.04034358 ... -0.07426413  0.09873463
  -0.6346708 ]]


In [96]:
q1_df_model1 = pd.DataFrame(q1_arr)
q2_df_model1 = pd.DataFrame(q2_arr)
X_model1 = pd.concat([q1_df_model1, q2_df_model1], axis=1)
y = df['is_duplicate']
y = np.array(y)
y_df = pd.DataFrame(y)
y_df.shape
df_model1 = X_model1.join(y_df, lsuffix='_left', rsuffix='_right')
df_model1.shape

(3000, 21)

In [98]:
y= df_model1['0_right']
X_train, X_test, y_train, y_test = train_test_split(df_model1, y, stratify = y, test_size = 0.3, random_state = 42)

#RandomForest

In [104]:
def rf_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Random Forest Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .3, random_state = 42)
    rf_param_grid = {'n_estimators':[20, 50, 80], 'min_samples_split':[3, 13], 'max_depth': [10, 30, None]}
    rf_clf = RandomForestClassifier(random_state = 42)
    
    rf_search = HalvingGridSearchCV(rf_clf, rf_param_grid, cv = splits, factor = 2, scoring = 'accuracy', 
                                        max_resources = 5, aggressive_elimination = False, verbose = 0)
    rf_search.fit(X_train, y_train)
    
    rf_model = rf_search.best_estimator_
    
    y_pred = rf_model.predict(X_test)
    
    log_loss = metrics.log_loss(y_test, y_pred)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred)
    recall = metrics.recall_score(y_test, y_pred)
    f1_score = metrics.f1_score(y_test, y_pred)
    print (rf_model)
    return accuracy, precision, recall, f1_score, log_loss, y_pred, rf_model

In [105]:
acc_rf, precision_rf, recall_rf, f1_score_rf, log_loss_rf, y_pred, berf  = rf_clf(X_train, X_test, y_train, y_test)

RandomForestClassifier(max_depth=10, min_samples_split=3, n_estimators=20,
                       random_state=42)


In [106]:
report_rf = classification_report(y_pred, y_test)
matrix_rf = confusion_matrix(y_pred, y_test)
print('Classification Report - RF\n', report_rf)
print()
print('Confusion Matrix - RF\n', matrix_rf)

Classification Report - RF
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       575
           1       1.00      1.00      1.00       325

    accuracy                           1.00       900
   macro avg       1.00      1.00      1.00       900
weighted avg       1.00      1.00      1.00       900


Confusion Matrix - RF
 [[575   0]
 [  0 325]]


# Logistic Regression Lemmatized

In [84]:
def lr_clf(X_train, X_test, y_train, y_test):
    '''Parameter tuning for Logistic Regression Classifier and model fit'''
    
    splits = ShuffleSplit(n_splits = 1, test_size = .2, random_state = 0)
    lr_param_grid = {'C':[100, 10, 1.0, 0.1, 0.01], 'solver':['sag', 'lbfgs']}
    lr_clf = LogisticRegression(penalty = 'l2', random_state = 0)
   
    lr_d2v_search = HalvingGridSearchCV(lr_clf, lr_param_grid, cv = splits, factor = 2, scoring = 'accuracy', verbose = 0)
    lr_d2v_search.fit(X_train, y_train)
    
    lr_d2v_model = lr_d2v_search.best_estimator_
    y_pred_lr_d2v = lr_d2v_model.predict(X_test)
    lr_loss_d2v = metrics.log_loss(y_test, y_pred_lr_d2v)
    lr_acc_d2v = metrics.accuracy_score(y_test, y_pred_lr_d2v)
    print (lr_d2v_model)
    print('log loss', lr_loss_d2v, '\nacc', lr_acc_d2v)
    return lr_acc_d2v, lr_loss_d2v, y_pred_lr_d2v   

In [85]:
lr_acc_d2v, lr_loss_d2v, y_pred_lr_d2v = lr_clf(X_train, X_test, y_train, y_test)

LogisticRegression(C=0.1, random_state=0, solver='sag')
log loss 9.992007221626413e-16 
acc 1.0


# SCV

In [111]:
from sklearn.svm import SVC

clf_svc = SVC(C=1.0, kernel='rbf', degree=3, gamma='scale', coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200
              , class_weight=None, verbose=False, max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=None)


clf_svc.fit(X_train, y_train)

y_pred = clf_svc.predict(X_test)
#y_pred

In [112]:
log_loss = metrics.log_loss(y_test, y_pred)
accuracy = metrics.accuracy_score(y_test, y_pred)
precision = metrics.precision_score(y_test, y_pred)
recall = metrics.recall_score(y_test, y_pred)
f1_score = metrics.f1_score(y_test, y_pred)


In [113]:
print(log_loss)
print(accuracy)
print(precision)
print(f1_score)

9.992007221626413e-16
1.0
1.0
1.0
